In [23]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine

In [6]:
data = pd.read_excel("data/global-superstore-data.xlsx", sheet_name="Orders")

data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [8]:
# Basic Transformation
data.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [10]:
#fxn to transform column
def transform_column_names(df:pd.DataFrame):
    '''
    This function transforms the column names by replacing the spaces by _ and removing all leading or lagging space
    '''
    print('Transforming Column names')
    df.columns = [column.strip().lower().replace(' ', '_').replace('-', '_') for column in df.columns]
    return df

In [11]:
#call function to create a new data frame call tr_data
tr_data = transform_column_names(data)

In [12]:
#checking the new column names
tr_data.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'city', 'state', 'country',
       'postal_code', 'market', 'region', 'product_id', 'category',
       'sub-category', 'product_name', 'sales', 'quantity', 'discount',
       'profit', 'shipping_cost', 'order_priority'],
      dtype='object')

In [7]:
#Normalization --> Breaking it all apart
#Orders
#Products
#Customers

In [22]:
print(f'Extracting & Transforming products... current length = {len(tr_data)}')

products_data = tr_data[['product_id', 'category', 'sub-category', 'product_name']].drop_duplicates()
print(f'Products Extraction & transformation completed... with {products_data.shape[0]} samples')

customers_data = tr_data[['customer_id', 'customer_name']].drop_duplicates()
print(f'Customers Extraction & transformation completed... with {customers_data.shape[0]} samples')


locations_data = tr_data[['city', 'state', 'country']].drop_duplicates()
print(f'Locations Extraction & transformation completed... with {locations_data.shape[0]} samples')


orders_data = tr_data[['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode','customer_id', 'product_id', 'segment','postal_code',
                       'market', 'region', 'sales', 'quantity', 'discount','profit', 'shipping_cost','order_priority']]
print(f'Orders Extraction & transformation completed... with {orders_data.shape[0]} samples')

Extracting & Transforming products... current length = 51290
Products Extraction & transformation completed... with 10768 samples
Customers Extraction & transformation completed... with 1590 samples
Locations Extraction & transformation completed... with 3812 samples
Orders Extraction & transformation completed... with 51290 samples
Clean Orders Extraction & transformation completed... with 25035 samples


In [38]:
# write dataframes to csv files

def write_to_csv(df:pd.DataFrame, name_of_file:str):
    print(f'>>> writing {name_of_file} data to intermiediate storage....')
    return df.to_csv(f'outputs/models/{name_of_file}.csv', index=False)

In [59]:
write_to_csv(products_data, 'products')
# write_to_csv(customers_data, 'customers')
# write_to_csv(locations_data, 'locations')
# write_to_csv(orders_data, 'orders')

>>> writing products data to intermiediate storage....


In [33]:
import os
from dotenv import load_dotenv
#load the dotenv file
load_dotenv(override=True)

True

In [61]:
#Access the variables
database_url = os.getenv('PG_URL')

In [60]:
os.listdir('outputs/models')

['products.csv']

In [64]:
engine = create_engine(database_url)

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres